In [11]:
import pandas as pd
import pymongo
from sqlalchemy import create_engine
import numpy as np
from config import pword, database

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
def randomIndex(low=0,high=10,size=10):
    index_list =[]
    while len(index_list) != size:
        num=np.random.randint(low, high,1)
        if num not in index_list:
            index_list.append(num[0])
    return sorted(list((index_list)))
        
def col_to_list(col):
    return list(col.find()), len(list(col.find()))

In [4]:
quotes_mdb=client.quotes_db
author_info_col=quotes_mdb.author_information_collection


author_list,len_author_list=col_to_list(author_info_col)

author_info_df=pd.DataFrame(author_list,index=randomIndex(100,300,len_author_list))


author_info_table = author_info_df[['name','born','description']].copy()
author_info_table.index.name='author_id'
author_info_table=author_info_table.drop_duplicates(subset=['name'])
author_info_table #sql table 1

,name,born,description
author_id,,,
100,Albert Einstein,"March 14, 1879 in Ulm, Germany","\n In 1879, Albert Einstein was born in..."
103,J.K. Rowling,"July 31, 1965 in Yate, South Gloucestershire, ...",\n See also: Robert GalbraithAlthough s...
105,Jane Austen,"December 16, 1775 in Steventon Rectory, Hampsh...",\n Jane Austen was an English novelist ...
107,Marilyn Monroe,"June 01, 1926 in The United States",\n Marilyn Monroe (born Norma Jeane Mor...
112,André Gide,"November 22, 1869 in Paris, France",\n André Paul Guillaume Gide was a Fren...
113,Thomas A. Edison,"February 11, 1847 in Milan, Ohio, The United S...",\n Thomas Alva Edison was an American i...
114,Eleanor Roosevelt,"October 11, 1884 in The United States",\n Anna Eleanor Roosevelt was an Americ...
117,Steve Martin,"August 14, 1945 in Waco, Texas, The United States","\n Stephen Glenn ""Steve"" Martin is an A..."
125,Bob Marley,"February 06, 1945 in Nine Mile, Saint Ann, Jam...","\n Robert ""Bob"" Nesta Marley OM was a J..."


In [5]:
tags_col=quotes_mdb.tags_collection
tags_list,len_tags_list=col_to_list(tags_col)
tags_df=pd.DataFrame(tags_list,index=randomIndex(301,500,len_tags_list))

tags_table = tags_df[['tag']].copy()
tags_table=tags_table.rename(columns={'tag':'tag_name'})
tags_table.index.name = 'tag_id'
tags_table #sql table 2

,tag_name
tag_id,
301,change
304,deep-thoughts
307,thinking
308,world
310,abilities
...,...
491,difficult
492,grown-ups
496,write


#### 

In [15]:
quotes_col=quotes_mdb.tag_relation_collection
quotes_list,len_quotes_list=col_to_list(quotes_col)
quotes_df= pd.DataFrame(quotes_list,index=randomIndex(501,700,len_quotes_list))
# quotes_df

quotes_and_tag_table = quotes_df[["quote_text",'tag']].copy()
quotes_and_tag_table=quotes_and_tag_table.rename(columns={'tag':'tag_list'})
quotes_and_tag_table.index.name = 'quote_id'
# quotes_and_tag_table

quoteid_tag=[]

for index, row in quotes_and_tag_table.iterrows():
    for tag in row['tag_list']:
        quoteid_tag.append((index,tag))
quote_tag_df=pd.DataFrame(quoteid_tag,columns=['quote_id','tag_name'])


# quote_tag_df

quotes_and_tag_relations_table=tags_table.reset_index().merge(quote_tag_df,on='tag_name')

quotes_and_tag_relations_table=quotes_and_tag_relations_table[['tag_id','quote_id']].copy()

quotes_and_tag_relations_table=quotes_and_tag_relations_table.set_index('tag_id') #sql table 3
quotes_and_tag_relations_table

,quote_id
tag_id,
301,502
304,502
307,502
307,638
308,502
...,...
491,686
492,686
496,686


In [13]:
quotes_table=quotes_and_tag_table[['quote_text']].copy()
quotes_table

,quote_text
quote_id,
502,“The world as we have created it is a process ...
503,"“It is our choices, Harry, that show what we t..."
506,“There are only two ways to live your life. On...
507,"“The person, be it gentleman or lady, who has ..."
508,"“Imperfection is beauty, madness is genius and..."
...,...
695,“You never really understand a person until yo...
696,“You have to write the book that wants to be w...
697,“Never tell the truth to people who are not wo...


In [14]:
# quotes_table

quotes_col=quotes_mdb.quotes_collection
quotes_and_author_df=pd.DataFrame(list(quotes_col.find()))
quotes_and_author_table=quotes_and_author_df[['quote_text','author_name']].copy().rename(columns={'author_name':'name'})

quotes_and_author_table=quotes_and_author_table.merge(author_info_table.reset_index(), on='name')
# quotes_and_author_table

quotes_and_author_table=(quotes_and_author_table[['quote_text','author_id']].copy()).merge(quotes_and_tag_table.reset_index(),on="quote_text").drop(['tag_list'],axis=1)

quotes_and_author_table=quotes_and_author_table.set_index('quote_id')

quotes_and_author_table

,quote_text,author_id
quote_id,,
502,“The world as we have created it is a process ...,100
506,“There are only two ways to live your life. On...,100
509,“Try not to become a man of success. Rather be...,100
531,"“If you can't explain it to a six year old, yo...",100
560,"“If you want your children to be intelligent, ...",100
...,...,...
691,“To die will be an awfully big adventure.”,286
692,“It takes courage to grow up and become who yo...,287
693,“But better to get hurt by the truth than comf...,288


In [12]:

                
rds_connection_string = f"postgres:{pword}@localhost:5432/{database}"#<=== username is postgres, password is your pword, and your database
engine = create_engine(f'postgresql://{rds_connection_string}')

# Confirm tables
engine.table_names()

['quote_tag_relation', 'tags', 'author_info', 'quotes']

In [27]:


author_info_table.to_sql(name='author_info',con=engine,if_exists='append',index=True)

quotes_and_author_table.to_sql(name='quotes',con=engine,if_exists='append',index=True)

tags_table.to_sql(name='tags',con=engine,if_exists='append',index=True)

quotes_and_tag_relations_table.to_sql(name='quote_tag_relation',con=engine,if_exists='append',index=True)